In [ ]:
import numpy as np
import polars as pl
import polars_plugin_option_pricing as m
print(f"polars_plugin_option_pricing version: {m.__version__}")

polars_plugin_option_pricing version: 0.1.0


In [2]:
def _range(e):
    start = e["start"]
    end = e["end"]
    n_step = e["n_step"]
    return np.linspace(start, end, n_step)

In [3]:
%%time

spots = {"start": 98, "end": 102, "n_step": 3}
strikes = {"start": 100, "end": 100, "n_step": 1}
mats = {"start": 1, "end": 10, "n_step": 1}
vols = {"start": 0.1, "end": 0.5, "n_step": 1}
rates = {"start": 0.00, "end": 0.04, "n_step": 1}
divs = {"start": 0.00, "end": 0.07, "n_step": 1}

spots = {"start": 85, "end": 115, "n_step": 25}
strikes = {"start": 85, "end": 115, "n_step": 100}
mats = {"start": 1, "end": 10, "n_step": 20}
vols = {"start": 0.1, "end": 0.5, "n_step": 5}
rates = {"start": 0.00, "end": 0.04, "n_step": 5}
divs = {"start": 0.00, "end": 0.07, "n_step": 8}

df_is_call = pl.DataFrame(data=[True], schema=["is_call"])
df_spot = pl.DataFrame(data=_range(spots), schema={"spot": pl.Float32})
df_strike = pl.DataFrame(data=_range(strikes), schema={"strike": pl.Float32})
df_mat = pl.DataFrame(data=_range(mats), schema={"mat": pl.Float32})
df_vol = pl.DataFrame(data=_range(vols), schema={"vol": pl.Float32})
df_rate = pl.DataFrame(data=_range(rates), schema={"rate": pl.Float32})
df_div = pl.DataFrame(data=_range(divs), schema={"div": pl.Float32})

CPU times: user 639 μs, sys: 1.01 ms, total: 1.65 ms
Wall time: 1.45 ms


In [4]:
%%time

df = (
    df_is_call.join(df_spot, how="cross")
    .join(df_strike, how="cross")
    .join(df_mat, how="cross")
    .join(df_vol, how="cross")
    .join(df_rate, how="cross")
    .join(df_div, how="cross")
)
print(df)

shape: (10_000_000, 7)
┌─────────┬───────┬────────┬──────┬─────┬──────┬──────┐
│ is_call ┆ spot  ┆ strike ┆ mat  ┆ vol ┆ rate ┆ div  │
│ ---     ┆ ---   ┆ ---    ┆ ---  ┆ --- ┆ ---  ┆ ---  │
│ bool    ┆ f32   ┆ f32    ┆ f32  ┆ f32 ┆ f32  ┆ f32  │
╞═════════╪═══════╪════════╪══════╪═════╪══════╪══════╡
│ true    ┆ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.0  │
│ true    ┆ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.01 │
│ true    ┆ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.02 │
│ true    ┆ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.03 │
│ true    ┆ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ 0.0  ┆ 0.04 │
│ …       ┆ …     ┆ …      ┆ …    ┆ …   ┆ …    ┆ …    │
│ true    ┆ 115.0 ┆ 115.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.03 │
│ true    ┆ 115.0 ┆ 115.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.04 │
│ true    ┆ 115.0 ┆ 115.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.05 │
│ true    ┆ 115.0 ┆ 115.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.06 │
│ true    ┆ 115.0 ┆ 115.0  ┆ 10.0 ┆ 0.5 ┆ 0.04 ┆ 0.07 │
└─────────┴───────┴────────┴──────┴─────┴──────┴──────┘
CPU times: user 190 ms, s

In [5]:
%%time

df = df.with_columns(
    output_bs=m.black_scholes("is_call", "spot", "strike", "mat", "vol", "rate", "div"),
).drop(["is_call"]).unnest("output_bs")
print(df)

shape: (10_000_000, 25)
┌───────┬────────┬──────┬─────┬───┬────────────┬────────────┬────────┬───────────┐
│ spot  ┆ strike ┆ mat  ┆ vol ┆ … ┆ rho        ┆ voma       ┆ payoff ┆ pv_payoff │
│ ---   ┆ ---    ┆ ---  ┆ --- ┆   ┆ ---        ┆ ---        ┆ ---    ┆ ---       │
│ f32   ┆ f32    ┆ f32  ┆ f32 ┆   ┆ f32        ┆ f32        ┆ f32    ┆ f32       │
╞═══════╪════════╪══════╪═════╪═══╪════════════╪════════════╪════════╪═══════════╡
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 40.805199  ┆ -0.846693  ┆ 0.0    ┆ 0.0       │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 37.432491  ┆ 2.514806   ┆ 0.0    ┆ 0.0       │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 34.109966  ┆ 12.325046  ┆ 0.0    ┆ 0.0       │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 30.869394  ┆ 27.908499  ┆ 0.0    ┆ 0.0       │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 27.740191  ┆ 48.265541  ┆ 0.0    ┆ 0.0       │
│ …     ┆ …      ┆ …    ┆ …   ┆ … ┆ …          ┆ …          ┆ …      ┆ …         │
│ 115.0 ┆ 115.0  ┆ 10.0 ┆ 0.5 ┆ … ┆ 180.00824  ┆ -92.713051 ┆ 0

In [6]:
%%time

df = df.with_columns(
    iv_output=m.implied_vol(
        "price",
        "spot",
        "strike",
        "mat",
        "rate",
        "div",
        iter=10,
        prec=1e-7,
        # method="Newton",
        method="Halley",
    )
)

CPU times: user 3.32 s, sys: 112 ms, total: 3.43 s
Wall time: 3.43 s


In [7]:
%%time

df = df.unnest("iv_output")
print(df)

shape: (10_000_000, 28)
┌───────┬────────┬──────┬─────┬───┬───────────┬─────────────┬──────────────┬──────────────┐
│ spot  ┆ strike ┆ mat  ┆ vol ┆ … ┆ pv_payoff ┆ vol_implied ┆ iter_implied ┆ prec_implied │
│ ---   ┆ ---    ┆ ---  ┆ --- ┆   ┆ ---       ┆ ---         ┆ ---          ┆ ---          │
│ f32   ┆ f32    ┆ f32  ┆ f32 ┆   ┆ f32       ┆ f32         ┆ u32          ┆ f32          │
╞═══════╪════════╪══════╪═════╪═══╪═══════════╪═════════════╪══════════════╪══════════════╡
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.0       ┆ 0.1         ┆ 2            ┆ -0.001411    │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.0       ┆ 0.1         ┆ 4            ┆ -0.000004    │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.0       ┆ 0.1         ┆ 2            ┆ 0.009552     │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.0       ┆ 0.1         ┆ 10           ┆ 0.000011     │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.0       ┆ 0.1         ┆ 3            ┆ 0.00001      │
│ …     ┆ …      ┆ …    ┆ …   ┆ … ┆ …         ┆ …       

In [8]:
%%time

df = df.with_columns(vol_prec=(pl.col("vol") - pl.col("vol_implied")).abs())
print(df)

dfs = df.select([f"{e}_implied" for e in ["vol", "iter", "prec"]] + ["vol_prec"])
print(dfs.describe())

shape: (10_000_000, 29)
┌───────┬────────┬──────┬─────┬───┬─────────────┬──────────────┬──────────────┬───────────┐
│ spot  ┆ strike ┆ mat  ┆ vol ┆ … ┆ vol_implied ┆ iter_implied ┆ prec_implied ┆ vol_prec  │
│ ---   ┆ ---    ┆ ---  ┆ --- ┆   ┆ ---         ┆ ---          ┆ ---          ┆ ---       │
│ f32   ┆ f32    ┆ f32  ┆ f32 ┆   ┆ f32         ┆ u32          ┆ f32          ┆ f32       │
╞═══════╪════════╪══════╪═════╪═══╪═════════════╪══════════════╪══════════════╪═══════════╡
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.1         ┆ 2            ┆ -0.001411    ┆ 1.9372e-7 │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.1         ┆ 4            ┆ -0.000004    ┆ 2.9802e-8 │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.1         ┆ 2            ┆ 0.009552     ┆ 1.0431e-7 │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.1         ┆ 10           ┆ 0.000011     ┆ 2.6077e-7 │
│ 85.0  ┆ 85.0   ┆ 1.0  ┆ 0.1 ┆ … ┆ 0.1         ┆ 3            ┆ 0.00001      ┆ 7.4506e-8 │
│ …     ┆ …      ┆ …    ┆ …   ┆ … ┆ …           ┆ …     